# Project 1

    We begin by loading the data

In [20]:

from preprocessing_functions import *
%matplotlib inline 
import numpy as np   # generic stuff
import matplotlib.pyplot as plt

#### REMOVE THIS LINE BEFORE SUBMISSION
import pandas as pd
#######################################################################

from lib.proj1_helpers import * #the helper provided for the project
from lib.costs import *

# choose which implementations you would like
from lib.implementations import *
#from implementations import * #our implementations of the functions done by us


import datetime
from helpers import * #helpers of exo 2
# Useful starting lines

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import operator

In [22]:
DATA_FOLDER = '../data/' # get rid of the ..

y_train, tx_train, ids_train = load_csv_data(DATA_FOLDER+'train.csv',sub_sample=False)

y_test, tx_test, ids_test = load_csv_data(DATA_FOLDER+'test.csv',sub_sample=False)
AAA = 0;

note that here we are only considering a sub_sample as the "True" value indicates

In [23]:
print(tx_train.shape)
print(tx_test.shape)
print(y_train.shape)

(250000, 30)
(568238, 30)
(250000,)


In [24]:
# illegal: 
pd.DataFrame(tx_train).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,...,-1.916,164.546,1.0,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,...,-2.186,260.414,1.0,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,...,0.060,86.062,0.0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,...,-0.871,53.131,0.0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


# Methods

### Functions for methods: 

In [15]:
OUT_FOLDER = 'output/'

def func_least_squares (y, tx, test_set, fct='mse'):
    #name = 'least_squares'
    w,loss = least_squares(y,tx,fct)
    #y_pred = predict_labels(w, test_set)
    #create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('least squares weights for loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

def func_GD (y, tx, test_set, max_iters, gamma, initial_w):
    w,loss = least_squares_GD(y, tx, initial_w, max_iters, gamma,fct='mse');
    #y_pred = predict_labels(w, test_set)
    #create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('GD: weights with loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

########################  RIDGE REGRESSION #######################################################
def func_ridge_regression (y, tx, test_set, lambda_):
    name = 'Ridge_regression'
    
    w,loss = ridge_regression(y, tx, lambda_=lambda_, fct='mse');
    
    y_pred = predict_labels(w, test_set)
    create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('Ridge regression: weights ;loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

## LEAST-SQUARES

In [ ]:
#todo: cross_validation su degree

In [18]:
def build_k_indices(y, k_fold, seed):
    """Build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation_least_square(y, tx, k_fold, degs, seed):
    """Finds the best degree for Least Squares method with cross validation."""
    # Get k'th subgroup in test, others in train
    k_indices = build_k_indices(y, k_fold, seed)
    
    # Initialisation of the matrices of accuracies for all the degrees and different folds
    accuracy_train = np.zeros([k_fold, len(degs)])
    accuracy_test = np.zeros([k_fold, len(degs)])

    # Loop over the different folds
    for k in range(k_fold):
        print('----- FOLD', k, '-----')
        k_index = k_indices[k]
        test_y = y[k_index]
        test_tx = tx[k_index,:]
        mask = np.ones(len(y), dtype=bool) # set all elements to True
        mask[k_index] = False              # set test elements to False
        train_tx = tx[mask,:]              # select only True elements (ie train elements)
        train_y = y[mask]

        len_init_data = 0

        # Loop over the different degrees to try
        for i, single_deg in enumerate(degs):
            print('++++ Deg =', single_deg)
            if i==0:
                # Preprocess the data (cleaning and adding features)
                train_tx, test_tx, len_init_data = prepare_data(train_tx, test_tx, single_deg);
            else:
                nb_cols_tx = train_tx.shape[1]
                # Add the supplementary powers of the features with respect to the previous iteration and standardize
                train_tx = add_powers(train_tx, single_deg, 0, len_init_data, features='x', current_max_deg=single_deg-1)
                test_tx = add_powers(test_tx, single_deg, 0, len_init_data, features='x', current_max_deg=single_deg-1)
                train_tx[:,nb_cols_tx:] = standardize(train_tx[:,nb_cols_tx:])[0]
                test_tx[:,nb_cols_tx:] = standardize(test_tx[:,nb_cols_tx:])[0]
                unique_cols = keep_unique_cols(train_tx)
                train_tx = train_tx[:,unique_cols]
                test_tx = test_tx[:,unique_cols]
                
            # Compute the weights with LS
            weights, loss = least_squares(train_y, train_tx, fct='mse');

            # Compute the predictions
            y_pred_train = predict_labels(weights, train_tx)
            y_pred_test = predict_labels(weights, test_tx);
            accuracy_train[k, i] = np.sum(y_pred_train == train_y)/len(train_y)
            accuracy_test[k, i] = np.sum(y_pred_test == test_y)/len(test_y)

    accuracies_test = np.mean(accuracy_test, axis=0); # mean on the k's
    accuracies_train = np.mean(accuracy_train, axis=0);

    max_index, acc_max = max(enumerate(accuracies_test), key=operator.itemgetter(1))
    print(accuracy_test)
    print(accuracies_test)
    print(k_fold)
    print(len(degs))
    print(max_index)
    print(acc_max)
    deg_best = degs[max_index]
    
    return deg_best, acc_max;

In [19]:
OUT_FOLDER = 'output/'
name = 'least_squares.csv'

degs=range(2,14)
k_fold=5
seed=1;

mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)
y_preds_train = np.zeros(len(y_train))
y_preds_test = np.zeros(tx_test.shape[0])


for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train = y_train[mask_jets_train[mask_jet_id]]
    
    best_deg, best_acc = cross_validation_least_square(y_single_jet_train, tx_single_jet_train, k_fold, degs,seed)
    print('Best degree = ', best_deg)
    
    tx_single_jet_train, tx_single_jet_test,len_init_data = prepare_data(tx_single_jet_train,tx_single_jet_test, best_deg)
    w, loss = func_least_squares(y=y_single_jet_train, tx=tx_single_jet_train, test_set=tx_single_jet_test, fct='mse')
    
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train)*100
    
    print('**** Jet ', mask_jet_id, '***** Accuracy jet', right_train)
    

create_csv_submission(ids_test, y_preds_test, OUT_FOLDER+name)
right_train_total = np.sum(y_train == y_preds_train)/len(y_train)*100
print('>>>>>>>> Accuracy TOTAL ',right_train_total)

**** Starting Jet  0 *****
----- FOLD 0 -----
++++ Deg = 2
++++ Deg = 3
++++ Deg = 4
++++ Deg = 5
++++ Deg = 6
++++ Deg = 7
++++ Deg = 8
++++ Deg = 9
++++ Deg = 10


KeyboardInterrupt: 

In [ ]:
len(y_preds_test[y_preds_test==1])

In [ ]:
len(y_preds_test[y_preds_test==-1])+len(y_preds_test[y_preds_test==1])==tx_test.shape[0]

In [ ]:
y_preds_test[:200]

## Gradient descent 

In [ ]:
#cross validation su gamma e degree 

In [25]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)



def cross_validation_GD(y, tx, k_fold, max_iters, degs,lambdas):            
    seed=1;
    
    # get k'th subgroup in test, others in train
    k_indices = build_k_indices(y, k_fold, seed)
    accuracy_train = np.zeros([k_fold, len(degs),len(gammas)])
    accuracy_test = np.zeros([k_fold, len(degs),len(gammas)])

    acc_max=0;
    deg_best=0;
    gammas_best=0;
               
    for k in range(k_fold):
        print('----- FOLD', k, '-----')
        k_index = k_indices[k]
        test_y = y[k_index]
        test_tx = tx[k_index,:]
        mask = np.ones(len(y), dtype=bool) # set all elements to True
        mask[k_index] = False              # set test elements to False
        train_tx = tx[mask,:]              # select only True elements (ie train elements)
        train_y = y[mask]

        len_init_data = 0

        for i, single_deg in enumerate(degs):
            print('++++ Deg =', single_deg)
            for j , single_gamma_ in  enumerate(gammas):
                print('++++ gamma =', single_gamma_)
                
                if i==0:
                    train_tx, test_tx, len_init_data = prepare_data(train_tx, test_tx, single_deg);
                else:
                    shape_tx=train_tx.shape[1];
                    train_tx = add_powers(train_tx, single_deg, 0, len_init_data, features='x', current_max_deg=single_deg-1)
                    test_tx = add_powers(test_tx, single_deg, 0, len_init_data, features='x', current_max_deg=single_deg-1)
                    train_tx[:,shape_tx:]=standardize(train_tx[:,shape_tx:])[0]
                initial_w = np.zeros(train_tx.shape[1]);
                
                weights,loss = least_squares_GD(train_y,train_tx, initial_w, max_iters, single_gamma_,fct='mse');

                # Compute the predictions
                y_pred_train = predict_labels(weights, train_tx);
                y_pred_test = predict_labels(weights, test_tx);
                accuracy_train[k, i,j] = np.sum(y_pred_train == train_y)/len(train_y)
                accuracy_test[k, i,j] = np.sum(y_pred_test == test_y)/len(test_y)
        
      
                
    accuracies_test=np.mean(accuracy_test, axis=0); # mean on the k's
    accuracies_train=np.mean(accuracy_train, axis=0);

 
    for i, single_deg in enumerate(degs):
         for j , single_gamma_ in  enumerate(gammas):
                if (accuracies_test[i,j]>acc_max):
                    deg_best=degs[i];
                    gammas_best=gammas[j];
                    acc_mac=accuracies_test[i,j];
    
    
                    
        
                
    return [deg_best, gamma_best, acc_max];




In [26]:
cubic=np.array([[1,1,1],[2,2,2]])
print(cubic)
print (cubic[:,1:].shape)

cubic[:,1:]=standardize(cubic[:,1:])[0];
print(cubic)




[[1 1 1]
 [2 2 2]]
(2, 2)
[[ 1 -1 -1]
 [ 2  1  1]]


In [ ]:

OUT_FOLDER = 'output/'
name = 'Gradient_descent.csv'
degs=range(4,8)
gammas=np.logspace(-4,-1,8);
k_fold=5
max_iters=200


mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_preds_train = np.zeros(len(y_train))
y_preds_test = np.zeros(tx_test.shape[0])



for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test =  tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train =   y_train[mask_jets_train[mask_jet_id]]
    
    
    
    [best_deg, gamma_best, acc_max] =cross_validation_GD(y_single_jet_train, tx_single_jet_train, k_fold, max_iters, degs,gammas)
    tx_single_jet_train, tx_single_jet_test,len_init_data = prepare_data(tx_single_jet_train,tx_single_jet_test, best_deg)
    
    initial_w = np.zeros(tx_single_jet_train.shape[1]);
    w, loss = func_GD (y=y_single_jet_train, tx=tx_single_jet_train, test_set=tx_single_jet_test, max_iters=max_iters,gamma=best_gamma, initial_w=initial_w);
    
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train )*100
    
    print('**** Jet ', mask_jet_id, '*****accuracy jet',right_train)
    

create_csv_submission(ids_test, y_preds_test, OUT_FOLDER+name)
right_train_total = np.sum(y_train == y_preds_train)/len(y_train)*100

print('>>>>>>>> Accuracy TOTAL ',right_train_total)

**** Starting Jet  0 *****
----- FOLD 0 -----
++++ Deg = 4
++++ gamma = 0.0001
Gradient Descent(0/199): loss=0.4999999999999887, w0=-6.887232304259569e-08, w1=-3.5851239558151825e-05
Gradient Descent(1/199): loss=0.4997886023020471, w0=-1.3279692118023493e-07, w1=-7.165426932699993e-05
Gradient Descent(2/199): loss=0.49957776994619885, w0=-1.9180268329462098e-07, w1=-0.00010740917970274563
Gradient Descent(3/199): loss=0.49936750087119885, w0=-2.459183816411012e-07, w1=-0.00014311606088238587
Gradient Descent(4/199): loss=0.49915779302441243, w0=-2.951726722743844e-07, w1=-0.000178775002864219
Gradient Descent(5/199): loss=0.498948644361784, w0=-3.395940954727797e-07, w1=-0.00021438609544834942
Gradient Descent(6/199): loss=0.49874005284779666, w0=-3.792110761609197e-07, w1=-0.0002499494282371909
Gradient Descent(7/199): loss=0.4985320164554317, w0=-4.1405192433098656e-07, w1=-0.00028546509063596823
Gradient Descent(8/199): loss=0.4983245331661278, w0=-4.4414483546245015e-07, w1=-0.000

Gradient Descent(80/199): loss=0.4847180205653432, w0=8.170770635895399e-06, w1=-0.0027551443760327927
Gradient Descent(81/199): loss=0.48454598484937883, w0=8.41834328295822e-06, w1=-0.002787367316371041
Gradient Descent(82/199): loss=0.48437437375865905, w0=8.66886758023028e-06, w1=-0.0028195487644615655
Gradient Descent(83/199): loss=0.4842031858079534, w0=8.92232282649081e-06, w1=-0.0028516887960879055
Gradient Descent(84/199): loss=0.4840324195179678, w0=9.1786884071243e-06, w1=-0.002883787486871045
Gradient Descent(85/199): loss=0.4838620734153176, w0=9.437943793801222e-06, w1=-0.0029158449122698147
Gradient Descent(86/199): loss=0.48369214603250016, w0=9.700068544159913e-06, w1=-0.0029478611475812944
Gradient Descent(87/199): loss=0.4835226359078691, w0=9.965042301489575e-06, w1=-0.0029798362679412105
Gradient Descent(88/199): loss=0.48335354158560695, w0=1.0232844794414407e-05, w1=-0.0030117703483243364
Gradient Descent(89/199): loss=0.48318486161569973, w0=1.050345583657885e-0

## Ridge regression   

In [ ]:
deg=5;
lambda_=1e-5
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)
for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test =  tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train =   y_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_train, tx_single_jet_test, len_init = prepare_data(tx_single_jet_train, tx_single_jet_test, deg);
    
    [w,loss]=func_ridge_regression (y=y_single_jet_train, tx=tx_single_jet_train, test_set=tx_single_jet_test, lambda_=lambda_);    
    
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train )*100
    
    print('**** Jet ', mask_jet_id, '*****accuracy jet',right_train)
    

    
right_train_total = np.sum(y_train == y_preds_train)/len(y_train )*100
print('>>>>>>>> Accuracy TOTAL ',right_train_total)

##  SGD

In [ ]:
def func_SGD (y, tx, test_set, max_iters, gamma, initial_w, batch_size):
    name = 'Stochastic_Gradient_descent'
    
    w,loss = least_squares_SGD(y, tx, initial_w, max_iters,\
                               gamma, batch_size)
    
    y_pred = predict_labels(w, test_set)
    create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('SGD: weights with loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

In [ ]:
deg=5;
lambda_=1e-5
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)
for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test =  tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train =   y_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_train, tx_single_jet_test, len_init = prepare_data(tx_single_jet_train, tx_single_jet_test, deg);
    
    initial_w = np.zeros(tx_single_jet_train.shape[1]);
    w, loss = func_SGD (y=y_single_jet_train, tx=tx_single_jet_train, test_set=tx_single_jet_test, max_iters=500,gamma=0.00005, initial_w=initial_w,\
                   batch_size=1);    
   
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train )*100
    
    print('**** Jet ', mask_jet_id, '*****accuracy jet',right_train)
    

    
right_train_total = np.sum(y_train == y_preds_train)/len(y_train )*100
print('>>>>>>>> Accuracy TOTAL ',right_train_total)






## LOGISTIC REGRESSION

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation_logistic_regression(y,tx, k_fold, max_iters, gammas):
    
    accuracies_test=np.zeros(len(gammas))
    accuracies_train=np.zeros(len(gammas))
    acc_max=0;
    gamma_best=0;
    
    for i, single_gamma in enumerate(gammas):
        
        seed=1;

        # get k'th subgroup in test, others in train
        k_indices = build_k_indices(y, k_fold, seed)
        accuracy_train = np.zeros(k_fold)
        accuracy_test = np.zeros(k_fold)


        for k in range(k_fold):
            #print('----- FOLD', k, '-----')
            k_index = k_indices[k]
            test_y = y[k_index]
            test_tx = tx[k_index,:]
            mask = np.ones(len(y), dtype=bool) # set all elements to True
            mask[k_index] = False              # set test elements to False
            train_tx = tx[mask,:]              # select only True elements (ie train elements)
            train_y = y[mask]
            initial_w = np.zeros(train_tx.shape[1]);
            weights,loss = logistic_regression(train_y, train_tx, initial_w, max_iters, single_gamma)
            # Compute the predictions
            y_pred_train = predict_labels(weights, train_tx)
            y_pred_test = predict_labels(weights, test_tx);
            accuracy_train[k] = np.sum(y_pred_train == train_y)/len(train_y)
            accuracy_test[k] = np.sum(y_pred_test == test_y)/len(test_y)
            
        accuracies_test[i]=np.mean(accuracy_test);
        accuracies_train[i]=np.mean(accuracy_train);
        
        print('GAMMA:', single_gamma, '---','ACCURANCY TEST:',accuracies_test[i], '---','ACCURANCY TRAIN:',accuracies_train[i])
        if (accuracies_test[i]>acc_max):
                gamma_best=gammas[i];
                acc_max=accuracies_test[i];
    
    return [gamma_best,acc_max]
               
        # Compute accuracy of the predictions

        

In [ ]:
def func_logistic (y, tx, test_set, max_iters, gamma, initial_w):
    name = 'Logistic regression'
    
    #w,loss = logistic_regression(y, tx, initial_w, max_iters, gamma)
    w,loss = logistic_regression(y, tx, initial_w, max_iters, gamma)
    
    loss=loss/len(y);
    y_pred = predict_labels(w, test_set)
    create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)  
    
    plt.plot(w, 'go')
    plt.title('logistic reg: weights with loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

In [ ]:
gammas=np.linspace(0.00001,0.9,10);
k_fold=3;
max_iters=1000;
deg=5;


mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)
for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test =  tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train =   y_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_train, tx_single_jet_test, len_init = prepare_data(tx_single_jet_train, tx_single_jet_test, deg);
    
    
    
    [best_gamma,acc_max]=cross_validation_logistic_regression(y_single_jet_train, tx_single_jet_train, k_fold, max_iters, gammas)
    
    print('BEST GAMMA:', best_gamma, '---','ACCURANCY:',acc_max)

    initial_w = np.zeros(tx_single_jet_train.shape[1]);
    w, loss = func_logistic (y=y_single_jet_train, tx=tx_single_jet_train, test_set=tx_single_jet_test, max_iters=1000,\
                         gamma=best_gamma, initial_w=initial_w);
    
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train )*100
    
    print('**** Jet ', mask_jet_id, '*****accuracy jet',right_train)
    

    
right_train_total = np.sum(y_train == y_preds_train)/len(y_train )*100
print('>>>>>>>> Accuracy TOTAL ',right_train_total)


    
    
    


## LOGISTIC REGULARIZED

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation_logistic_regression_reg(y,tx, k_fold, max_iters, gammas,lambdas):
    
    acc_max=0;
    accuracies_test=np.zeros([len(gammas),len(lambdas)]);
    
    accuracies_train=np.zeros([len(gammas),len(lambdas)]);
    
    
    for i, single_gamma in enumerate(gammas):
        for j, single_lambda in enumerate(lambdas):
            seed=1;
        
            # get k'th subgroup in test, others in train
            k_indices = build_k_indices(y, k_fold, seed)
            accuracy_train = np.zeros(k_fold)
            accuracy_test = np.zeros(k_fold)


            for k in range(k_fold):
                #print('----- FOLD', k, '-----')
                k_index = k_indices[k]
                test_y = y[k_index]
                test_tx = tx[k_index,:]

                mask = np.ones(len(y), dtype=bool) # set all elements to True
                mask[k_index] = False              # set test elements to False
                train_tx = tx[mask,:]              # select only True elements (ie train elements)
                train_y = y[mask]
                initial_w = np.zeros(train_tx.shape[1]);
                weights,loss = reg_logistic_regression(train_y, train_tx,single_lambda, initial_w, max_iters, single_gamma)
                # Compute the predictions
                y_pred_train = predict_labels(weights, train_tx)
                y_pred_test = predict_labels(weights, test_tx)
                predictions=True;
                accuracy_train[k] = np.sum(y_pred_train == train_y)/len(train_y)
                accuracy_test[k] = np.sum(y_pred_test == test_y)/len(test_y)
                
                
            accuracies_test[i,j]= np.mean(accuracy_test);
            accuracies_train[i,j]= np.mean(accuracy_train);
            print('GAMMA', single_gamma, '---','LAMBDA', single_lambda, '---ACCURANCY TEST:',accuracies_test[i,j],'---ACCURANCY TRAIN:',accuracies_train[i,j])
            
            if (accuracies_test[i,j]>acc_max):
                gamma_best=gammas[i];
                lambda_best=lambdas[j];
                acc_max=accuracies_test[i,j];
            
            
    return [gamma_best,lambda_best,acc_max]
     
    
    
            # Compute accuracy of the predictions

            

In [ ]:
def func_logistic_reg (y, tx, lambda_, test_set, max_iters, gamma, initial_w):
    name = 'Logistic regression regularized'
    
    #w,loss = logistic_regression(y, tx, initial_w, max_iters, gamma)
    w,loss = reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)
    
    loss=loss/len(y);
    y_pred = predict_labels(w, test_set)
    create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('logistic reg: weights with the normalized log-like:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

In [ ]:
#test cross_validation

gammas=np.linspace(0.000001,0.0001,2);
lambdas=np.linspace(0.001,0.3,2);
k_fold=6;
max_iters=50;

deg=2;

mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)
for mask_jet_id in range(len_mask):
    print('**** Starting Jet ', mask_jet_id, '*****')
    tx_single_jet_train = tx_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_test =  tx_test[mask_jets_test[mask_jet_id]]
    y_single_jet_train =   y_train[mask_jets_train[mask_jet_id]]
    tx_single_jet_train, tx_single_jet_test, len_init
    = prepare_data(tx_single_jet_train, tx_single_jet_test, deg);
    
    [gamma_best,lambda_best,acc_max]= cross_validation_logistic_regression_reg(y_single_jet_train, tx_single_jet_train, k_fold, max_iters, gammas,lambdas)
    print('BEST GAMMA', gamma_best, '---','BEST LAMBDA', lambda_best, '---ACCURANCY:',acc_max)
    
    initial_w = np.zeros(tx_single_jet_train.shape[1]);
    w, loss = func_logistic_reg (y=y_single_jet_train, tx=tx_single_jet_train, lambda_=lambda_best, test_set=tx_single_jet_test , max_iters=max_iters,\
                         gamma=gamma_best, initial_w=initial_w);


    
    y_pred_train = predict_labels(w, tx_single_jet_train)
    y_pred_test = predict_labels(w, tx_single_jet_test)
    y_preds_train[mask_jets_train[mask_jet_id]] = y_pred_train
    y_preds_test[mask_jets_test[mask_jet_id]] = y_pred_test
    
    right_train = np.sum(y_single_jet_train == y_pred_train)/len(y_single_jet_train )*100
    
    print('**** Jet ', mask_jet_id, '*****accuracy jet',right_train)
    

    
right_train_total = np.sum(y_train == y_preds_train)/len(y_train )*100
print('>>>>>>>> Accuracy TOTAL ',right_train_total)


    
    
